# M1 Model Checkpoint Analysis

Here I ensure the reproducability of results from training experiments produced 
using the scripts in `src/execute_m1.py`, `src/train_m1.py`. 
 - Load model.
 - Compute performance metrics. 
 - Visualize 10-frame autoencoding results. 

The first run of model checkpoints are in `training/2022_10_25/`

In [1]:
## Import Box 
import os 
import sys 
import random
import pathlib
import itertools
import collections
import math

import tensorflow as tf 
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
# Some modules to display an animation using imageio.
import imageio

2022-10-25 16:46:51.677371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 16:46:51.803006: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-25 16:46:51.828294: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-25 16:46:52.393239: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
## GPU Setup
physical_devices = tf.config.list_physical_devices("GPU")
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

## 0: Getting the Dataset

_Copy-pasted from `execute_m1.py`_

In [4]:
## Acquiring the dataset!
sys.path.append("../src")
import video_loader as vl
import video_preprocess as vp

# Meta/constants -- TODO: These should be commandline arguments. 
DATA_FOLDER = "../datasets/downloads"
num_frames = 10
output_size = (120, 180)

patch_height = 16
patch_width = 16
patch_duration = 3

batch_size = 10
num_prefetch = 4

# Fourier feature codes 
k_space = 15
mu_space = 20 
k_time = 64 
mu_time = 200

# Mp4 list: 
mp4_list = os.listdir(DATA_FOLDER)

print("\n\n\n")
print("\t=========================")
print("\t=== DATASET META INFO ===")
print("\t=========================")
print("num_frames: ", num_frames)
print("batch_size: ", batch_size)
print("output_size: ", output_size)
print("Patch h/w/d: ", patch_height, patch_width, patch_duration)
print("k, mu for space, time: ", (k_space, mu_space), (k_time, mu_time))
print("mp4_list[:10] -- ", mp4_list[:10])


## Creating the generator
print("\n\n\n")
print("\t========================")
print("\t=== DATASET CREATION ===")
print("\t========================")
def generate_video_tensors():
	""" This is a generator for raw video tensors of shape 
	[num_frames, height, width, channels]. 

	It uses global variables defined above under "meta/constants". These 
	will be commandline arguments in the future. 
	"""
	_mp4_list = mp4_list
	_DATA_FOLDER = DATA_FOLDER
	_output_size = output_size
	_num_frames = num_frames
	while True: 
		for fname in _mp4_list:
			try:
				retval = vl.get_single_video_tensor(os.path.join(_DATA_FOLDER, fname), _num_frames, output_size=_output_size)
			except:
				continue
			if type(retval) == np.ndarray and retval.shape[0] == _num_frames:
				yield np.expand_dims(retval, axis=0)

videoset = tf.data.Dataset.from_generator(generate_video_tensors, output_signature=tf.TensorSpec(shape=[1, num_frames, *output_size, 3], dtype=tf.float32))
print(videoset)

def show_nn_sq(video_tensor, n=3, outname="breh.png"):
	n2 = n*n
	fig, axs = plt.subplots(nrows=n, ncols=n, sharex=True, sharey=True)

	_nframes = video_tensor.shape[0]
	frame_inc = _nframes//(n2)

	for i in range(n):
		for j in range(n):
			full_idx = i*n + j

			axs[i,j].imshow(video_tensor[frame_inc*full_idx,:,:,:])
			axs[i,j].set_title(f"Frame {frame_inc*full_idx}")

	fig.suptitle(f"Video Survey over {_nframes} Frames")
	plt.savefig(outname)
	plt.show()

"""
out_test = "debug/test_from_videoset.png"
print(f"Showing off an element of the videodataset(generator) in `{out_test}`")	
for element in videoset:
	show_nn_sq(tf.squeeze(element), outname=out_test)
	break
"""

print("Making patches from Videoset...")
PatchSet = vp.make_patchset(videoset, patch_duration, patch_height, patch_width)

print("Making the flat patch set...")
FlatPatchSet = vp.patch_to_flatpatch(PatchSet, batch_size=1)

print("Adding codes to the PatchSet...")
CodedPatchedSet = PatchSet.map(lambda x: vp.add_spacetime_codes(x, 
		k_space=k_space, mu_space=mu_space, k_time=k_time, mu_time=mu_time))

print("Flattening the coded + patched dataset...")
FlatCodedPatchedSet = vp.patch_to_flatpatch(CodedPatchedSet, batch_size=1)
FlatCodedPatchedSet = FlatCodedPatchedSet.map(lambda x: tf.squeeze(x))
FlatCodedPatchedSet = FlatCodedPatchedSet.batch(batch_size)
FlatCodedPatchedSet = FlatCodedPatchedSet.prefetch(num_prefetch)

cnt = 0
for el in FlatCodedPatchedSet: 
	print(" ** Shape of FlatCodedPatchedSet element: ", el.shape)
	cnt += 1
	if cnt == 2:
		break

print("Done getting datasets setup!")





	=== DATASET META INFO ===
num_frames:  10
batch_size:  10
output_size:  (120, 180)
Patch h/w/d:  16 16 3
k, mu for space, time:  (15, 20) (64, 200)
mp4_list[:10] --  ['H1iWETEMl7c.mp4', 'PmKbAgKa_V8.mp4', 'zymMiwUtbqE.mp4', 'jpeB76IUI9c.mp4', '5d-u6-UzbLk.mp4', 'm-EUUkPuocE.mp4', '_LUKHNsSndk.mp4', 'xp5sV_1SSKw.mp4', 'JXtr3GbDYHY.mp4', '3SeOIr2YisM.mp4']




	=== DATASET CREATION ===
<FlatMapDataset element_spec=TensorSpec(shape=(1, 10, 120, 180, 3), dtype=tf.float32, name=None)>
Making patches from Videoset...
Making the flat patch set...
Adding codes to the PatchSet...


2022-10-25 16:49:46.018291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 16:49:46.942589: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17631 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:1a:00.0, compute capability: 8.6
2022-10-25 16:49:46.943129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22279 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:68:00.0, compute capability: 8.6


Flattening the coded + patched dataset...
 ** Shape of FlatCodedPatchedSet element:  (10, 231, 2495)
 ** Shape of FlatCodedPatchedSet element:  (10, 231, 2495)
Done getting datasets setup!


## 1: Initializing Perceiver AE Model

_Random weights, checkpoint weights will be inserted next_

In [33]:
## Setting up the model
print("\t========================")
print("\t=== SETTING UP MODEL ===")
print("\t========================")

import m1
## Setting up the component modules of the top-level PAE model.
# Encoder 
n_encoder_blocks = 3
p_droptoken = 0.3
re_droptoken = True
encoder_tfres = False
enc_nheads = 15
enc_keydim = 15
enc_mhadropout = 0.0
test_encoder = m1.PAE_Encoder(n_encoder_blocks, p_droptoken=p_droptoken, re_droptoken=re_droptoken, tfblock_residual=encoder_tfres, n_heads=enc_nheads, key_dim=enc_keydim, mha_dropout=enc_mhadropout)

# Latent evolver 
n_latentev_blocks = 3
latent_distinct_blocks = False
latent_residual = False
latent_nheads = 15
latent_keydim = 15
latent_mhadropout=0.0
test_latent_ev = m1.PAE_Latent_Evolver(n_latentev_blocks, distinct_blocks=latent_distinct_blocks, tfblock_residual=latent_residual, n_heads=latent_nheads, key_dim = latent_keydim, mha_dropout=latent_mhadropout)


# decoder
output_patch_dim = 2304
n_decoder_blocks = 3
expansion_block_num = 2
decoder_tfres = False
dec_nheads = 15
dec_keydim = 15
dec_mhadropout = 0.0
test_decoder = m1.PAE_Decoder(output_patch_dim, n_decoder_blocks, expansion_block_num, tfblock_residual=decoder_tfres, n_heads=dec_nheads, key_dim=dec_keydim, mha_dropout=dec_mhadropout)

# loss function
mse = tf.keras.losses.MeanSquaredError()

## Instantiating the PAE model!
perceiver_ae = m1.PerceiverAE(mse, test_encoder, test_latent_ev, test_decoder, code_dim=191)

perceiver_ae.reset_latent()


print("Done setting up perceiver_ae model!")

	=== SETTING UP MODEL ===
Done setting up perceiver_ae model!


## 2: Obtaining Checkpoint Information

Question: Can we insert checkpoint weights even if we initialize with a different 
number of encoder blocks? 

 - No error upon loading in weights!
 - No error when loading weights into model with different # latent blocks!
 - Cumulative loss over 3 "exposures" = 0.19 -- in line with test performance!

In [34]:
checkpoint_path = "../training/2022_10_25/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'../training/2022_10_25/cp-0450.ckpt'

In [41]:
perceiver_ae.load_weights(latest)

In [42]:
## Calling the model on some data
ele = None
for element in FlatCodedPatchedSet:
	ele = element
	break

ele.shape

TensorShape([10, 231, 2495])

In [43]:
loss1 = perceiver_ae(ele, reset_latent=True)

In [44]:
loss2 = perceiver_ae(ele)

In [45]:
loss3 = perceiver_ae(ele)

In [46]:
print("Loss 1: ", loss1)
print("Loss 2: ", loss2)
print("Loss 3: ", loss3)
print("\n\tSUM: ", loss1+loss2+loss3)

Loss 1:  tf.Tensor(0.11647031, shape=(), dtype=float32)
Loss 2:  tf.Tensor(0.039892886, shape=(), dtype=float32)
Loss 3:  tf.Tensor(0.03990355, shape=(), dtype=float32)

	SUM:  tf.Tensor(0.19626674, shape=(), dtype=float32)


In [48]:
perceiver_ae.encoder.p_droptoken

0.3